In [ ]:
code = '''
import time
import sys

huge_output = 'fdsfsfdsffsfasfd' * 100

print('dfsfdfasfs', flush = True, end = '')
time.sleep(1)
#for i in range(0, 30):
#    print(huge_output, flush = True)
print('==endline', flush = True)
time.sleep(1)
print("fatal error", file=sys.stderr, flush = True)
time.sleep(3)
print('bccasfdsfdsfds', flush = True)
x = input('Input:')
#x = 'xxx'
print(f'x={x}', flush = True)
time.sleep(1)
print('ending...', flush = True)
exit(-1)
'''

In [ ]:
# import ipywidgets as widgets
# from ipywidgets import AppLayout, Button, Layout
# from ipywidgets import GridspecLayout

# b = widgets.Button(
#     description='Clear Output',
#     disabled=False,
#     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     icon='check'
# )

# display(b)

# async def reenable_button(task, bt):
#     await task
#     bt.disabled = False

# def call(bt):
#     task = asyncio.ensure_future(test())
#     bt.disabled = True
#     asyncio.ensure_future(reenable_button(task, bt))

# b.on_click(call)

In [ ]:
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import GridspecLayout
import logging
    
class ProcUI:
    _title_string = None
    _status_string = 'Not Started'
    _out = None
    
    _display_widges = None
    _all_widges = None
    
    _mode = 'Unknown'
            
    def __init__(
        self, 
        title = 'Executing Process'
    ):
        self._display_widges = []
        self._all_widges = []
        
        self._title_string = title

        self._out = widgets.Output(
            layout=Layout(height='300px', overflow_y='auto', border='1px solid black'),
            scroll = True
        )

        self._header_grid = self._init_header()
                
        self._display_widges.append(self._header_grid)
        self._display_widges.append(self._out)
        
        self._all_widges.extend(self._display_widges)
    
    
    def _reset_build_title(self):
        self._header.value = f'<b>{self._title_string}</b> Mode: <b>{self._mode}</b> ' \
                             f'Status: <b>{self._status_string}</b>'

    def _init_header(self):     
        self._header = widgets.HTML(
            value = '',
            layout=Layout(height='20', width='auto')
        )
        
        self._reset_build_title()
        
        self._clear_output_button = widgets.Button(
            description = 'Clear Output',
            disabled = True,
            button_style = 'info', # 'success', 'info', 'warning', 'danger' or ''
            icon = 'trash'
        )
        
        @self._out.capture(clear_output=True)
        def clear_output_cb(event):
            pass

        self._clear_output_button.on_click(clear_output_cb)
                
        header_grid = GridspecLayout(1, 2)
        header_grid[0, 0] = self._header
        header_grid[0, 1] = self._clear_output_button

        self._all_widges.append(self._header)        
        self._all_widges.append(self._clear_output_button)
        
        return header_grid

    def display(self):
        for w in self._display_widges:
            if w != None:
                display(w)
            
    def notify_status(self, status):
        self._status_string = f'[{status}]'
        self._reset_build_title()
            
    def destroy(self):
        for w in self._all_widges:
            if w != None:
                w.close()
        self._all_widges = []
        self._display_widges = []
    
    def notify_new_data(self, data):
        if self._out != None:
            with self._out:
                print(data, end='')
            
    def _clear_output(self):
        if self._out != None:
            self._out.clear_output()
    
    def set_mode(self, mode):
        self._mode = mode
        self._reset_build_title()

        if mode == 'Background':
            self._clear_output_button.disabled = False

class ProcWrapper:
    def __init__(
        self,
        executable, 
        *args,
        stdout_cb = None,
        stderr_cb = None
    ):
        """
        Callback will need to have 2 args (process object, decoded utf-8 data)
        """
        self._exe = executable
        self._args = args
        self._std_cb = stdout_cb
        self._err_cb = stderr_cb
    
    async def start(self):
        logging.info(f'ProcWrapper::start {self._exe} {self._args}')
        
        self._proc = await asyncio.create_subprocess_exec(
            self._exe, *self._args,
            stdout = asyncio.subprocess.PIPE,
            stderr = asyncio.subprocess.STDOUT,
            stdin = asyncio.subprocess.PIPE
        )
        
        logging.info(f'START PID: {self._proc.pid}')
            
        fullcontent = ''
        
        while True:
            data = await self._proc.stdout.read(1024)
            if data == b'':
                break
            data = data.decode('utf-8')
            # print(line, end='')
            self._std_cb(self._proc, data)
            fullcontent += data
            if fullcontent.endswith('Input:'):
                self._proc.stdin.write('myinput\n'.encode())
        
        ec = await self._proc.wait()

        logging.info('Error Code %d' % ec)
                
    async def wait(self):
        if self._proc:
            await self._proc.wait()

    def start_background(self):
        asyncio.ensure_future(
            self.start()
        )

class UIProcWrapper(ProcWrapper):
    _ui = None
    _exit_on_finish = False
    
    def __init__(
        self,
        executable, 
        *args, 
        stdout_cb = None,
        stderr_cb = None,
        exit_on_finish = False
    ):
        self._ui = ProcUI()
        self._exit_on_finish = exit_on_finish
        
        def overwritten_stdout_cb(proc, line):
            self._ui.notify_new_data(line)
            if stdout_cb != None:
                stdout_cb(proc, line)
            
        super().__init__(
            executable, 
            *args,
            stdout_cb = overwritten_stdout_cb, 
            stderr_cb = stderr_cb
        )
        
        self._mode = 'Foreground'
    
    async def start(self):
        self._ui.set_mode(self._mode)
        self._ui.display()
        
        try:
            self._ui.notify_status('Running')
            
            await super().start()

            self._ui.notify_status('Finished')
        finally:
            if self._exit_on_finish:
                self._ui.destroy()
    
    def start_background(self):
        self._mode = 'Background'
        super().start_background()

In [ ]:
# await ProcWrapper(sys.executable, '-c', code).start()
# ProcWrapper(sys.executable, '-c', code).start_background()

#await UIProcWrapper(sys.executable, '-c', code).start()
UIProcWrapper(sys.executable, '-c', code).start_background()

#await UIProcWrapper('ls').start()